# NTRU using sage

In [32]:
# Define a class Zx, which is a polynomial in x with integer coefficients
Zx.<x> = ZZ[]

In [33]:
f = Zx([3,1,4])

In [34]:
n = 743
d = 495
q = 2048
n = 7
d = 5
q = 256

## Arthimetic Element

### Cyclic convolution
This is the multiplication operation used in NTRU. It is the same as polynomial multiplication but reduces the output "modulo x^n-1": this means replacing x^n with 1, replacing x^(n+1) with x, replacing x^(n+2) with x^2, etc.

In [35]:
def convolution(f, g):
    return (f * g) % (x^n-1)

### Modular reduction
Mathematicians normally define reduction to produce outputs between 0 and q-1, but balancedmod instead produces outputs **between -q/2 and q/2**: more precisely, between -q/2 and q/2-1 if q is even, or between -(q-1)/2 and (q-1)/2 if q is odd.

In [36]:
def balancedmod(f, q):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    return Zx(g)

### Random polynomials with d nonzero coefficients
returns an n-coefficient polynomial where exactly d coefficients are nonzero (d are nonzero, the other n-d are all zero).

In [37]:
def randomdpoly():
    assert d <= n
    result = n*[0]
    for j in range(d):
        while True:
            r = randrange(n)
            if not result[r]: 
                break
        result[r] = 1 - 2*randrange(2)
    return Zx(result)

### Division modulo primes
invertmodprime computes a reciprocal of a polynomial modulo x^n-1 modulo p. There are two inputs: an n-coefficient polynomial f; and a prime number p (for example, 3). The output is an n-coefficient polynomial g so that convolution(f,g) is 1+p*u for some polynomial u. invertmodprime raises an exception if no such g exists.

In [38]:
def invertmodprime(f, p):
    T = Zx.change_ring(Integers(p)).quotient(x^n-1)
    return Zx(lift(1/T(f)))

### Division modulo powers of 2
Just like invertmodprime above, except that the second input q is 2 or 4 or 8 or 16 or ..

In [39]:
def invertmodpowerof2(f, q):
    assert q.is_power_of(2)
    g = invertmodprime(f,2)
    while True:
        r = balancedmod(convolution(g,f),q)
        if r == 1:
            return g
        g = balancedmod(convolution(g, 2-r), q)

## NTRU Encryption

### Key Generation
This returns an NTRU public key h and a corresponding secret key f,f3.

In [40]:
def keypair():
    while True:
        try:
            f = randomdpoly()
            f3 = invertmodprime(f,3)
            fq = invertmodpowerof2(f,q)
            break
        except:
            pass
    g = randomdpoly()
    publickey = balancedmod(3*convolution(fq, g), q)
    secretkey = f,f3
    return publickey, secretkey

In [41]:
def randommsg():
    result = list(randrange(3) - 1 for j in range(n))
    return Zx(result)

In [42]:
def encrypt(message, publickey):
    r = randomdpoly()
    return balancedmod(convolution(publickey, r) + message, q)

In [43]:
def decrypt(ciphertext, secretkey):
    f,f3 = secretkey
    a = balancedmod(convolution(ciphertext, f), q)
    return balancedmod(convolution(a,f3),3)

## Test

In [44]:
for tests in range(1000):
    publickey,secretkey = keypair()
    m = randommsg()
    c = encrypt(m,publickey)
    print m == decrypt(c,secretkey)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [45]:
n = 7
f = Zx([1,-1,1,1,-1])
f3 = invertmodpowerof2(f, 256)
f3

-126*x^6 + 87*x^5 + 58*x^4 - 47*x^3 + 54*x^2 + 36*x - 61

In [50]:
f = Zx([1,-1,0,-1,0,1,-1])
f3 = invertmodprime(f,3)
f3

2*x^6 + 2*x^5 + 2*x^3 + 2*x^2 + x + 2